In [1]:
import pandas as pd
import numpy as np

In [2]:
tagall = pd.read_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\thyroidtag.pkl")

In [4]:
thyroid = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201014\甲状腺dict.xlsx")

In [6]:
tagidlist = thyroid.loc[thyroid.logic.isin (['A','a']),'tag_id']

In [7]:
tagall_ = pd.merge(tagall,tagidlist)

In [8]:
ordercode_list = []
taglist = []
for order_code in np.unique(tagall_.order_code):
    taglist_ = tagall_.loc[tagall_.order_code == order_code, 'tag_id'].values.tolist() 
    ordercode_list.append(order_code)
    taglist.append(taglist_)
    
df_ = pd.DataFrame({"order_code": ordercode_list, "taglist":taglist})

In [13]:
df_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\A.xlsx")

In [21]:
df_AB = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\AB.xlsx",index_col = 0)

In [31]:
df_abbr = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\abbr.xlsx",index_col = 0)

In [38]:
df_A = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\A.xlsx",index_col = 0)

In [66]:
df_.taglist.values.tolist()

AttributeError: 'DataFrame' object has no attribute 'taglist'

In [42]:
df_A.taglist.values.tolist()

["['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0010']",
 "['OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0010', 'OBDIM0074Tag0003']",
 "['OBDIM0074Tag0010', 'OBDIM0074Tag0005', 'OBDIM0074Tag0015', 'OBDIM0074Tag0001']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0005', 'OBDIM0074Tag0011', 'OBDIM0074Tag0001']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010', 'OBDIM0074Tag0002']",
 "['OBDIM0074Tag0010', 'OBDIM0074Tag0006']",
 "['OBDIM0074Tag0005', 'OBDIM0074Tag0015', 'OBDIM0074Tag0011']",
 "['OBDIM0074Tag0004', 'OBDIM0074Tag0010']",
 "['OBDIM0074Tag0004', 'OBDIM0074Ta

In [45]:
# 构造数据
def loadDataSet(data):
    DataSet = data.taglist.values.tolist()
    return DataSet
 
# 将所有元素转换为frozenset型字典，存放到列表中
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # 使用frozenset是为了后面可以将这些值作为字典的键
    return list(map(frozenset, C1))  # frozenset一种不可变的集合，set可变集合
 
# 过滤掉不符合支持度的集合
# 返回 频繁项集列表retList 所有元素的支持度字典
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):   # 判断can是否是tid的《子集》 （这里使用子集的方式来判断两者的关系）
                if can not in ssCnt:    # 统计该值在整个记录中满足子集的次数（以字典的形式记录，frozenset为键）
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []        # 重新记录满足条件的数据值（即支持度大于阈值的数据）
    supportData = {}    # 每个数据值的支持度
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData # 排除不符合支持度元素后的元素 每个元素支持度
 
# 生成所有可以组合的集合
# 频繁项集列表Lk 项集元素个数k  [frozenset({2, 3}), frozenset({3, 5})] -> [frozenset({2, 3, 5})]
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk): # 两层循环比较Lk中的每个元素与其它元素
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]  # 将集合转为list后取值
            L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()        # 这里说明一下：该函数每次比较两个list的前k-2个元素，如果相同则求并集得到k个元素的集合
            if L1==L2:
                retList.append(Lk[i] | Lk[j]) # 求并集
    return retList  # 返回频繁项集列表Ck
 
# 封装所有步骤的函数
# 返回 所有满足大于阈值的组合 集合支持度列表
def apriori(dataSet, minSupport):
    D = list(map(set, dataSet)) # 转换列表记录为字典  [{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
    C1 = createC1(dataSet)      # 将每个元素转会为frozenset字典    [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    L1, supportData = scanD(D, C1, minSupport)  # 过滤数据
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):    # 若仍有满足支持度的集合则继续做关联分析
        Ck = aprioriGen(L[k-2], k)  # Ck候选频繁项集
        Lk, supK = scanD(D, Ck, minSupport) # Lk频繁项集
        supportData.update(supK)    # 更新字典（把新出现的集合:支持度加入到supportData中）
        L.append(Lk)
        k += 1  # 每次新组合的元素都只增加了一个，所以k也+1（k表示元素个数）
    return L, supportData
 
dataSet = loadDataSet(df_)
L,suppData = apriori(dataSet,0.5)

In [46]:
suppData

{frozenset({'OBDIM0074Tag0004'}): 0.5648376070780526,
 frozenset({'OBDIM0074Tag0010'}): 0.8853614279886909,
 frozenset({'OBDIM0074Tag0003'}): 0.032858368615756826,
 frozenset({'OBDIM0074Tag0001'}): 0.0646617810473605,
 frozenset({'OBDIM0074Tag0005'}): 0.19596866076828942,
 frozenset({'OBDIM0074Tag0015'}): 0.08511386493747626,
 frozenset({'OBDIM0074Tag0011'}): 0.06378968393512723,
 frozenset({'OBDIM0074Tag0002'}): 0.013109588848409829,
 frozenset({'OBDIM0074Tag0006'}): 0.08033139690264865,
 frozenset({'OBDIM0074Tag0016'}): 0.02306837522681558,
 frozenset({'OBDIM0074Tag0012'}): 0.0031508024700040793,
 frozenset({'OBDIM0074Tag0022'}): 0.004585542880452365,
 frozenset({'OBDIM0074Tag0024'}): 0.0006048415455811402,
 frozenset({'OBDIM0074Tag0033'}): 8.439649473225212e-05,
 frozenset({'OBDIM0074Tag0025'}): 0.00378377618049597,
 frozenset({'OBDIM0074Tag0023'}): 9.846257718762748e-05,
 frozenset({'OBDIM0074Tag0030'}): 7.033041227687677e-05,
 frozenset({'OBDIM0074Tag0034'}): 0.0009283614420547734

In [47]:
# 获取关联规则的封装函数
def generateRules(L, supportData, minConf):  # supportData 是一个字典
    bigRuleList = []
    for i in range(1, len(L)):  # 从为2个元素的集合开始
        for freqSet in L[i]:
            # 只包含单个元素的集合列表
            H1 = [frozenset([item]) for item in freqSet]    # frozenset({2, 3}) 转换为 [frozenset({2}), frozenset({3})]
            # 如果集合元素大于2个，则需要处理才能获得规则
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf) # 集合元素 集合拆分后的列表 。。。
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList
 
# 对规则进行评估 获得满足最小可信度的关联规则
def calcConf(freqSet, H, supportData, brl, minConf):
    prunedH = []  # 创建一个新的列表去返回
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]  # 计算置信度
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH
 
# 生成候选规则集合
def rulesFromConseq(freqSet, H, supportData, brl, minConf):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): # 尝试进一步合并
        Hmp1 = aprioriGen(H, m+1) # 将单个集合元素两两合并
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
dataSet = loadDataSet(df_)
L,suppData = apriori(dataSet,0.3)
rules = generateRules(L,suppData,0.3)

frozenset({'OBDIM0074Tag0004'}) --> frozenset({'OBDIM0074Tag0010'}) conf: 0.9811734236477738
frozenset({'OBDIM0074Tag0010'}) --> frozenset({'OBDIM0074Tag0004'}) conf: 0.6259631730295665


In [48]:
pd.DataFrame(rules)

,0,1,2
0,(OBDIM0074Tag0004),(OBDIM0074Tag0010),0.981173
1,(OBDIM0074Tag0010),(OBDIM0074Tag0004),0.625963


,OBDIM0074Tag0001,OBDIM0074Tag0002,OBDIM0074Tag0003,OBDIM0074Tag0004,OBDIM0074Tag0005,OBDIM0074Tag0006,OBDIM0074Tag0010,OBDIM0074Tag0011,OBDIM0074Tag0012,OBDIM0074Tag0015,...,OBDIM0074Tag0023,OBDIM0074Tag0024,OBDIM0074Tag0025,OBDIM0074Tag0029,OBDIM0074Tag0030,OBDIM0074Tag0031,OBDIM0074Tag0032,OBDIM0074Tag0033,OBDIM0074Tag0034,OBDIM0074Tag0037
0,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71088,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71089,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71090,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71091,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

df_AB = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201027\A.xlsx",index_col = 0)
AB_list = []
for item in df_AB.taglist:
    item_ = eval(item)
    AB_list.append(item_)
AB_list


te = TransactionEncoder()
df_tf = te.fit_transform(AB_list)
df = pd.DataFrame(df_tf,columns=te.columns_)

frequent_itemsets = apriori(df,min_support=0.01,use_colnames= True)
rules = association_rules(frequent_itemsets,metric = 'confidence',min_threshold = 0.15)
rules.rename(columns = {'antecedents':'from','consequents':'to','support':'sup','confidence':'conf'},inplace = True)
rules.sort_values(by = ['sup','conf'],ascending = (False,False))
rules

,from,to,antecedent support,consequent support,sup,conf,lift,leverage,conviction
0,(OBDIM0074Tag0001),(OBDIM0074Tag0004),0.064662,0.564838,0.030650,0.474005,0.839188,-0.005873,0.827312
1,(OBDIM0074Tag0001),(OBDIM0074Tag0005),0.064662,0.195969,0.020508,0.317163,1.618439,0.007837,1.177487
2,(OBDIM0074Tag0001),(OBDIM0074Tag0010),0.064662,0.885361,0.053057,0.820535,0.926780,-0.004192,0.638780
3,(OBDIM0074Tag0002),(OBDIM0074Tag0010),0.013110,0.885361,0.010254,0.782189,0.883468,-0.001353,0.526321
4,(OBDIM0074Tag0003),(OBDIM0074Tag0004),0.032858,0.564838,0.010071,0.306507,0.542646,-0.008488,0.627493
5,(OBDIM0074Tag0003),(OBDIM0074Tag0010),0.032858,0.885361,0.032141,0.978168,1.104823,0.003049,5.250896
6,(OBDIM0074Tag0005),(OBDIM0074Tag0004),0.195969,0.564838,0.033027,0.168533,0.298374,-0.077663,0.523367
7,(OBDIM0074Tag0006),(OBDIM0074Tag0004),0.080331,0.564838,0.021198,0.263877,0.467173,-0.024177,0.591154
8,(OBDIM0074Tag0010),(OBDIM0074Tag0004),0.885361,0.564838,0.554204,0.625963,1.108218,0.054118,1.163421
9,(OBDIM0074Tag0004),(OBDIM0074Tag0010),0.564838,0.885361,0.554204,0.981173,1.108218,0.054118,6.089188


In [5]:
frequent_itemsets

,support,itemsets
0,0.064662,(OBDIM0074Tag0001)
1,0.013110,(OBDIM0074Tag0002)
2,0.032858,(OBDIM0074Tag0003)
3,0.564838,(OBDIM0074Tag0004)
4,0.195969,(OBDIM0074Tag0005)
5,0.080331,(OBDIM0074Tag0006)
6,0.885361,(OBDIM0074Tag0010)
7,0.063790,(OBDIM0074Tag0011)
8,0.085114,(OBDIM0074Tag0015)
9,0.023068,(OBDIM0074Tag0016)


In [3]:
df_AB

,order_code,taglist
0,000004527245,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
1,000010487220,['OBDIM0074Tag0010']
2,000034935327,['OBDIM0074Tag0010']
3,000044110416,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
4,000067663112,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
...,...,...
71088,999950213802,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
71089,999972061818,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
71090,999986953057,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"
71091,999990770784,"['OBDIM0074Tag0004', 'OBDIM0074Tag0010']"


In [4]:
AB_list

[['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0010'],
 ['OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0010', 'OBDIM0074Tag0003'],
 ['OBDIM0074Tag0010',
  'OBDIM0074Tag0005',
  'OBDIM0074Tag0015',
  'OBDIM0074Tag0001'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0005', 'OBDIM0074Tag0011', 'OBDIM0074Tag0001'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010', 'OBDIM0074Tag0002'],
 ['OBDIM0074Tag0010', 'OBDIM0074Tag0006'],
 ['OBDIM0074Tag0005', 'OBDIM0074Tag0015', 'OBDIM0074Tag0011'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM0074Tag0010'],
 ['OBDIM0074Tag0004', 'OBDIM

In [2]:
df

,OBDIM0074Tag0001,OBDIM0074Tag0002,OBDIM0074Tag0003,OBDIM0074Tag0004,OBDIM0074Tag0005,OBDIM0074Tag0006,OBDIM0074Tag0010,OBDIM0074Tag0011,OBDIM0074Tag0012,OBDIM0074Tag0015,...,OBDIM0074Tag0023,OBDIM0074Tag0024,OBDIM0074Tag0025,OBDIM0074Tag0029,OBDIM0074Tag0030,OBDIM0074Tag0031,OBDIM0074Tag0032,OBDIM0074Tag0033,OBDIM0074Tag0034,OBDIM0074Tag0037
0,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71088,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71089,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71090,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71091,False,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
